# (EX) Analyzing an article through PySpark

In [1]:
import os
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-11.0.2"
os.environ["SPARK_HOME"] = r"C:\Program Files\Spark\spark-3.5.5-bin-hadoop3"

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
                     .appName("Analyzing an unknown article.")
                     .getOrCreate())


In [4]:
spark

In [5]:
sc = spark.sparkContext

In [6]:
## documentation
spark.read??

Type:        property
String form: <property object at 0x00000000204B7600>
Source:     
# spark.read.fget
@property
def read(self) -> DataFrameReader:
    """
    Returns a :class:`DataFrameReader` that can be used to read data
    in as a :class:`DataFrame`.

    .. versionadded:: 2.0.0

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    Returns
    -------
    :class:`DataFrameReader`

    Examples
    --------
    >>> spark.read
    <...DataFrameReader object ...>

    Write a DataFrame into a JSON file and read it back.

    >>> import tempfile
    >>> with tempfile.TemporaryDirectory() as d:
    ...     # Write a DataFrame into a JSON file
    ...     spark.createDataFrame(
    ...         [{"age": 100, "name": "Hyukjin Kwon"}]
    ...     ).write.mode("overwrite").format("json").save(d)
    ...
    ...     # Read the JSON file as a DataFrame.
    ...     spark.read.format('json').load(d).show()
    +---+------------+
    |age|        name|
    +---+------------+
 

In [7]:
file_path = r'C:\Users\moses\OneDrive - Northwestern University\classes\active\dataeng-300-2025sp\de300-2025sp-notes\notes\dataset\pope.txt'

In [8]:
article = spark.read.text(file_path)

In [9]:
article

DataFrame[value: string]

In [10]:
article.printSchema()

root
 |-- value: string (nullable = true)



In [11]:
article.select(article.value)

DataFrame[value: string]

In [13]:
article.show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                        |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
from pyspark.sql.functions import col

In [ ]:

article.select(article.value)
article.select(article['value'])
article.select(col('value'))
article.select('value')

In [14]:
from pyspark.sql.functions import col, split

lines = article.select(
    split(col('value'), " ").alias('line')
)

In [16]:
lines.printSchema()

root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [15]:
lines.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|line                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
+-----------------------------------------------------------------

In [17]:
lines

DataFrame[line: array<string>]

In [18]:
from pyspark.sql.functions import explode

words = lines.select(explode(col("line")).alias('word'))

In [20]:
words.printSchema()

root
 |-- word: string (nullable = false)



In [22]:
from pyspark.sql.functions import lower

words_lower = words.select(lower(col("word")).alias('word_lower'))

In [23]:
words_lower.show(10)

+----------+
|word_lower|
+----------+
|      pope|
|       leo|
|       xiv|
|      lays|
|       out|
|    vision|
|        of|
|    papacy|
|       and|
|identifies|
+----------+
only showing top 10 rows



In [24]:
from pyspark.sql.functions import regexp_extract

words_clean = words_lower.select(
    regexp_extract(col("word_lower"), r"(\W+)?([a-z]+)", 2).alias("word_clean")
)

In [25]:
words_clean.show(10)

+----------+
|word_clean|
+----------+
|      pope|
|       leo|
|       xiv|
|      lays|
|       out|
|    vision|
|        of|
|    papacy|
|       and|
|identifies|
+----------+
only showing top 10 rows



In [26]:
words_nonull = words_clean.where(col("word_clean") != "")

words_nonull.show(100)

+------------+
|  word_clean|
+------------+
|        pope|
|         leo|
|         xiv|
|        lays|
|         out|
|      vision|
|          of|
|      papacy|
|         and|
|  identifies|
|          ai|
|          as|
|           a|
|        main|
|   challenge|
|         for|
|    humanity|
|          by|
|      nicole|
|    winfield|
|     updated|
|          pm|
|         cdt|
|         may|
|     vatican|
|        city|
|          ap|
|        pope|
|         leo|
|         xiv|
|        laid|
|         out|
|         the|
|      vision|
|          of|
|         his|
|      papacy|
|    saturday|
| identifying|
|  artificial|
|intelligence|
|          as|
|         one|
|          of|
|         the|
|        most|
|    critical|
|     matters|
|      facing|
|    humanity|
|         and|
|      vowing|
|          to|
|    continue|
|        with|
|        some|
|          of|
|         the|
|        core|
|  priorities|
|          of|
|        pope|
|     francis|
|         

In [27]:
groups = words_nonull.groupBy(col("word_clean"))

In [28]:
groups

GroupedData[grouping expressions: [word_clean], value: [word_clean: string], type: GroupBy]

In [29]:
counts = groups.count()

In [32]:
counts.orderBy('count', ascending=False).show(10)

+----------+-----+
|word_clean|count|
+----------+-----+
|       the|  138|
|        of|   62|
|        in|   57|
|       and|   50|
|        to|   42|
|       leo|   36|
|         a|   31|
|      pope|   30|
|   vatican|   26|
|       his|   26|
+----------+-----+
only showing top 10 rows



In [33]:
import pyspark.sql.functions as F

counts = (
    spark.read.text(file_path)
     .select(F.split(F.col('value'), ' ').alias('line'))
     .select(F.explode(F.col('line')).alias('word'))
     .select(F.lower(F.col('word')).alias('word'))
     .select(F.regexp_extract(F.col('word'), r"(\W+)?([a-z]+)", 2).alias('word'))
     .where(F.col('word') != "")
     .groupby('word')
     .count()
)

In [34]:
counts.show(10)

+------------+-----+
|        word|count|
+------------+-----+
|        some|    2|
|   traveling|    2|
|         few|    1|
|    medicare|    1|
|       vocal|    1|
|    received|    1|
|    armangue|    2|
|conversation|    1|
|     explain|    1|
|    hometown|    1|
+------------+-----+
only showing top 10 rows



In [35]:
cleaned = (
    spark.read.text(file_path)
     .select(F.split(F.col('value'), ' ').alias('line'))
     .select(F.explode(F.col('line')).alias('word'))
     .select(F.lower(F.col('word')).alias('word'))
     .select(F.regexp_extract(F.col('word'), r"(\W+)?([a-z]+)", 2).alias('word'))
     .where(F.col('word') != "")
)

In [47]:
cleaned.printSchema()

root
 |-- word: string (nullable = false)



In [43]:
lengths = cleaned.select(
    F.length(
        F.col('word')
    ).alias('length')
)

In [48]:
lengths.printSchema()

root
 |-- length: integer (nullable = false)



In [49]:
lengths.show(20)

+------+
|length|
+------+
|     4|
|     3|
|     3|
|     4|
|     3|
|     6|
|     2|
|     6|
|     3|
|    10|
|     2|
|     2|
|     1|
|     4|
|     9|
|     3|
|     8|
|     2|
|     6|
|     8|
+------+
only showing top 20 rows



In [50]:
lengths.groupby('length').count().orderBy('length', ascending=True).show(30)

+------+-----+
|length|count|
+------+-----+
|     1|   35|
|     2|  313|
|     3|  392|
|     4|  238|
|     5|  157|
|     6|  118|
|     7|  167|
|     8|  139|
|     9|   82|
|    10|   62|
|    11|   26|
|    12|   16|
|    13|    8|
|    14|    3|
|    15|    1|
+------+-----+



In [51]:
cleaned.show(10)

+----------+
|      word|
+----------+
|      pope|
|       leo|
|       xiv|
|      lays|
|       out|
|    vision|
|        of|
|    papacy|
|       and|
|identifies|
+----------+
only showing top 10 rows



In [55]:
letters = cleaned.select(
    F.explode(F.split(F.col('word'), ""))
)

In [61]:
letters.groupby('col').count().orderBy('col').show(26)

+---+-----+
|col|count|
+---+-----+
|  a|  807|
|  b|   72|
|  c|  355|
|  d|  338|
|  e| 1005|
|  f|  208|
|  g|  157|
|  h|  388|
|  i|  706|
|  j|   10|
|  k|   32|
|  l|  364|
|  m|  215|
|  n|  619|
|  o|  657|
|  p|  259|
|  q|    3|
|  r|  489|
|  s|  527|
|  t|  741|
|  u|  188|
|  v|  141|
|  w|   97|
|  x|   30|
|  y|  137|
|  z|   15|
+---+-----+

